<img height="250" src="https://i.postimg.cc/gjptBxF4/logo-gas-removebg-preview.png" width="250"/>

In [1]:
import os

import pandas as pd
from warnings import filterwarnings

filterwarnings("ignore")

# Importando dados climáticos e fenológicos
path = os.path.join(os.getcwd(), "data", "DADOS_ARTIGO_AMENDOIM.xlsx")
clima = pd.read_excel(path, "Dados Climáticos")
fenologia = pd.read_excel(path, "Fenologia")

In [2]:
# Restringindo valores de "clima" para ≥ 2023-11-23 00:00:00 e ≤ 2024-04-30 00:00:00
clima = clima.loc[(clima.Dia >= "2023-11-23 00:00:00") & (clima.Dia <= "2024-04-30 00:00:00")].reset_index(drop=True)

In [3]:
# Excluindo a primeira linha
fenologia = fenologia.iloc[:,0:]

# transformando a primeira linha em colunas
cols = fenologia.iloc[0,:].values
fenologia.columns = cols
fenologia = fenologia.iloc[1:,:].reset_index(drop=True)

In [4]:
# Renomeando valores inteiros de cultivar e repetição C1 - C₂₂ e B1 - B3
rep_mapping = {1: "B1", 2: "B2", 3: "B3"} # Dicionários de mapeamento
cultivar_mapping = {i: f"C{i}" for i in range(1, 23)}

# Aplicando os mapeamentos
fenologia['Rep'] = fenologia['Rep'].map(rep_mapping)
fenologia['Cultivar'] = fenologia['Cultivar'].map(cultivar_mapping)

In [5]:
# Convertendo datas para a mesma base
cols = fenologia.columns[2:]
for name in cols:
    fenologia[str(name)] = fenologia[str(name)].apply(pd.to_datetime)

In [6]:
from util.methods import create_dataframes

# Lista de estádios fenológicos
estadios = ["S", "VE", "V0", "V1", "V2", "V3", "V4", "V5", "V6", 
            "R1", "R2", "R3", "R4", "R5", "R6", "R7", "R8"]

# Listando cultivares
cultivares = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
              'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
              'C21', 'C22']

# Lista de repetições
repeticao = ['B1', 'B2', 'B3']

dfs = create_dataframes(clima, fenologia, cultivares, repeticao, estadios)
final_df = pd.concat(dfs, ignore_index=True)

In [7]:
from util.methods import fill_none_with_previous_value

final_df = fill_none_with_previous_value(final_df)

Calculando Graus dias

In [8]:
# Definindo colunas de Graus dias (DG) e Somatório (SDG)
final_df['DG'] = None
final_df['SDG'] = None
Tb = 13 # °C
Tmax = 33 # °C

# Criando iteração para calcular os dados das colunas
for i in final_df.index:
    if (final_df.at[i, 'T2M'] < Tb or 
            final_df.at[i, 'T2M'] > Tmax):
        final_df.at[i, 'DG'] = 0
    else:
        final_df.at[i, 'DG'] = max(0, final_df.at[i, 'T2M'] - Tb)
    if i > 0 and final_df.at[i, 'Rep'] == final_df.at[i-1, 'Rep']:
        # Continua o somatório se "Rep" for igual ao anterior
        final_df.at[i, 'SDG'] = final_df.at[i-1, 'SDG'] + final_df.at[i, 'DG']
    else:
        # Reinicia o somatório se "Rep" mudar
        final_df.at[i, 'SDG'] = final_df.at[i, 'DG']
        
# Convertendo dados numéricos do dataframe para float
final_df['SDG'] = final_df['SDG'].astype(float)
final_df["DG"] = final_df["DG"].astype(float)

Adicionando variáveis climáticas

In [9]:
from util.methods import timestamp_to_dj, dr, conversor_coord, Qo, fotop, par, etp_camargo, balanco_hidrico
from numpy import mean

lat = conversor_coord(21,14,5, hem="S")
lon = conversor_coord(48,17,9, long="W")
CAD = (0.3799-0.297)*1.13*300

# Santos et al. (2003)
angA = mean([0.2449, 0.2636, 0.3129, 0.2996, 0.2634])
angB = mean([0.4866, 0.4652, 0.3996, 0.4351, 0.4775])

final_df['NDJ'] = final_df['Dia'].apply(timestamp_to_dj)
final_df['DR'] = final_df['NDJ'].apply(dr)
final_df['Qo'] = final_df.apply(lambda x: Qo(lat, x['NDJ']), axis=1)
final_df['Fotop'] = final_df.apply(lambda x: fotop(lat, x['NDJ']), axis=1)
final_df['PAR'] = final_df['Qg'].apply(par)
final_df['ETp'] = final_df.apply(lambda x: etp_camargo(lat, x['NDJ'], x['T2M']), axis=1) # Camargo 1971

<img src="images\img.png"/>
<br>fonte: Vieira (2019). 
Disponível em: https://hdl.handle.net/11449/183552 
<br>

# Balanço hídrico

## CAD (mm) = ($\theta_{cc}$-$\theta_{pmp}$)$\times$Ds$\times$Z$\times$10 <br>
$\theta_{cc}$ = cm$^{3}$ cm$^{-3}$ <br>
$\theta_{pmp}$ = cm$^{3}$ cm$^{-3}$ <br>
Ds = g cm$^{-3}$ <br>
Z = cm (amendoim = 30 cm)

In [10]:
recorte = final_df.loc[(final_df['Cultivar']=="C1")&(final_df['Rep']=="B1"),:]
recorte = balanco_hidrico(recorte, CAD)

In [11]:
recorte.drop(columns=['PS', 'TMAX', 'TMIN', 'T2M', 'URMAX', 'URMIN', 'UR2M',
       'NDH > 90%', 'PREC', 'W2M', 'Rajada Máxima', 'Qg', 'Cultivar', 'Rep',
       'Estadio', 'DG', 'SDG', 'NDJ', 'DR', 'Qo', 'Fotop', 'PAR', 'ETp'], inplace=True)

final_df = final_df.merge(recorte, on='Dia', how='left')

In [12]:
# DC = Total de dias de ciclo
final_df['DC'] = None

# Pegar todas as combinações únicas de 'Cultivar' e 'Rep'
unique_combinations = final_df[['Cultivar', 'Rep']].drop_duplicates()

# Loop por cada combinação e calcular 'DC'
for _, row in unique_combinations.iterrows():
    cultivar = row['Cultivar']
    rep = row['Rep']

    # Filtrar dados para a combinação atual de 'Cultivar' e 'Rep'
    mask = (final_df['Cultivar'] == cultivar) & (final_df['Rep'] == rep)
    subset_df = final_df.loc[mask]

    # Encontrar a primeira ocorrência de 'S' e 'R8'
    start_date = subset_df.loc[subset_df['Estadio'] == 'S', 'Dia'].min()
    end_date = subset_df.loc[subset_df['Estadio'] == 'R8', 'Dia'].min()

    # Calcular a diferença de dias entre 'S' e 'R8'
    if pd.notna(start_date) and pd.notna(end_date) and start_date <= end_date:
        cycle_days = (end_date - start_date).days
        final_df.loc[mask, 'DC'] = cycle_days

In [13]:
final_df = final_df[['Dia','Cultivar', 'Rep','Estadio',"DC",
'PS','TMAX','TMIN','T2M','URMAX','URMIN','UR2M','NDH > 90%','PREC',"ETp",'W2M','Rajada Máxima','DG',
'NDJ', 'DR', 'Qo', 'Qg','Fotop', 'PAR', 
'P-ETP', 'NAC', 'ARM', 'ALT', 'ETR', 'DEF', 'EXC', 'SDG',]]

In [14]:
resultado = (final_df.groupby(["Cultivar","Rep","DC"]).agg({
    'PS':               "mean",
    'TMAX':             "mean",
    'TMIN':             "mean",
    'T2M':              "mean",
    'URMAX':            "mean",
    'URMIN':            "mean",
    'UR2M':             "mean",
    'NDH > 90%':        "mean",
    'PREC':             "sum",
    "ETp":              "mean",
    'W2M':              "mean",
    'Rajada Máxima':    "mean",
}).sort_values(by=['Cultivar'], ascending=True).reset_index())

In [15]:
# Saving dataframe
with pd.ExcelWriter(path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
    final_df.to_excel(writer, sheet_name='Dados_finais', index=False)
    
with pd.ExcelWriter(path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
    resultado.to_excel(writer, sheet_name='medias', index=False)